In [5]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, get_feature_names

In [8]:
data =pd.read_csv('ratings.csv')
sparse_features = ['userId', 'movieId','timestamp']
target = ['rating']
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
data

,userId,movieId,rating,timestamp
0,0,1,3.5,340880
1,0,28,3.5,340785
2,0,31,3.5,340801
3,0,46,3.5,340790
4,0,49,3.5,340774
...,...,...,...,...
1048570,7119,163,5.0,494540
1048571,7119,247,4.0,494524
1048572,7119,254,5.0,494519
1048573,7119,255,4.0,494545


In [11]:
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

[SparseFeat(name='userId', vocabulary_size=7120, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x0000014FA4AAC988>, embedding_name='userId', group_name='default_group', trainable=True),
 SparseFeat(name='movieId', vocabulary_size=14026, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x0000014FA4ACA388>, embedding_name='movieId', group_name='default_group', trainable=True),
 SparseFeat(name='timestamp', vocabulary_size=822889, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x0000014FA4AC2B08>, embedding_name='timestamp', group_name='default_group', trainable=True)]

In [12]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}
train_model_input

{'userId': array([3177,  421, 2516, ...,  909, 5453, 1394], dtype=int64),
 'movieId': array([  93,  981, 9674, ..., 1130, 1873, 3580], dtype=int64),
 'timestamp': array([653387, 521753, 569943, ..., 611421, 105896, 188159], dtype=int64)}

In [17]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task ='regression')
model.compile('adam', 'mse', metrics=['mse'],)
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2)
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

Train on 671088 samples, validate on 167772 samples


d:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
d:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


671088/671088 [==============================] - 114s 170us/sample - loss: 0.9439 - mse: 0.9374 - val_loss: 0.7479 - val_mse: 0.7345
test RMSE 0.8587782018658834
